In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score

In [2]:
# aapl_day = yf.download("AAPL", start="1980-12-08", end=None)
# aapl_day.to_pickle('./data/aapl_day.pkl')
# amd_day = yf.download("AMD", start="1980-03-17", end=None)
# amd_day.to_pickle('./data/amd_day.pkl')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
# aapl_hour = yf.download(tickers="AAPL",
#                        period="2y",
#                        interval="1h"
#                       )
# aapl_hour.to_pickle('./data/aapl_hr.pkl')

In [4]:
def compute_up_down(begin, end):
    if abs((begin - end) / begin) <= 0.007:
        return 0
    elif (begin - end) / begin > 0.007:
        return 1
    else:
        return 2
    
def compute_volume_per_dollar(vol, begin, end):
    if begin - end == 0:
        return vol
    else:
        return vol / abs(begin - end)
    
def compute_pct_change(num1, num2, dem):
    if dem == 0:
        return 0
    else:
        return (num2 - num1) / dem

In [5]:
a = pd.read_pickle('./data/amd_day.pkl')
a = a.rename(columns={'Adj Close': 'ac'
                      , 'Open': 'o'
                      , 'Close': 'c'
                      , 'High': 'h'
                      , 'Low': 'l'
                      , 'Volume': 'v'
                     })

In [6]:
a['dow'] = a.index.weekday

a['vpd_hlc'] = a.apply(lambda row: compute_volume_per_dollar(row['v'], row['h'], row['l']), axis=1)  # volume per dollar high low change
a['vpd_opac'] = a.apply(lambda row: compute_volume_per_dollar(row['v'], row['o'], row['c']), axis=1)  # volume per dollar open close absolute change

a['ac1'] = a['ac'].shift(1)
a['ac3'] = a['ac'].shift(3)
a['ac5'] = a['ac'].shift(5)
a['ac8'] = a['ac'].shift(8)
a['ac13'] = a['ac'].shift(13)
a['ac21'] = a['ac'].shift(21)

a['mu_rol3'] = a['ac'].rolling(window=3).mean()
a['mu_rol5'] = a['ac'].rolling(window=5).mean()
a['mu_rol8'] = a['ac'].rolling(window=8).mean()
a['mu_rol13'] = a['ac'].rolling(window=13).mean()
a['mu_rol21'] = a['ac'].rolling(window=21).mean()

a['std_rol3'] = a['ac'].rolling(window=3).std()
a['std_rol5'] = a['ac'].rolling(window=5).std()
a['std_rol8'] = a['ac'].rolling(window=8).std()
a['std_rol13'] = a['ac'].rolling(window=13).std()
a['std_rol21'] = a['ac'].rolling(window=21).std()

a['hl_pct'] = a.apply(lambda row: compute_pct_change(row['l'], row['h'], row['o']), axis=1)
a['oc_pct'] = a.apply(lambda row: compute_pct_change(row['o'], row['c'], row['o']), axis=1)

a['ac1_pct'] = a['ac1'] - a['ac'] / a['ac']
a['ac3_pct'] = a['ac3'] - a['ac'] / a['ac']
a['ac5_pct'] = a['ac5'] - a['ac'] / a['ac']
a['ac8_pct'] = a['ac8'] - a['ac'] / a['ac']
a['ac13_pct'] = a['ac13'] - a['ac'] / a['ac']
a['ac21_pct'] = a['ac21'] - a['ac'] / a['ac']

a.dropna(axis=0, inplace=True)

a['ac1_up_dwn'] = a.apply(lambda row: compute_up_down(row['ac1'], row['ac']), axis=1)
a['ac3_up_dwn'] = a.apply(lambda row: compute_up_down(row['ac3'], row['ac']), axis=1)
a['ac5_up_dwn'] = a.apply(lambda row: compute_up_down(row['ac5'], row['ac']), axis=1)
a['ac8_up_dwn'] = a.apply(lambda row: compute_up_down(row['ac8'], row['ac']), axis=1)
a['ac13_up_dwn'] = a.apply(lambda row: compute_up_down(row['ac13'], row['ac']), axis=1)
a['ac21_up_dwn'] = a.apply(lambda row: compute_up_down(row['ac21'], row['ac']), axis=1)

a['ac1_pred'] = a['ac1_up_dwn'].shift(-1)
a['ac3_pred'] = a['ac3_up_dwn'].shift(-3)
a['ac5_pred'] = a['ac5_up_dwn'].shift(-5)
a['ac8_pred'] = a['ac8_up_dwn'].shift(-8)
a['ac13_pred'] = a['ac13_up_dwn'].shift(-13)
a['ac21_pred'] = a['ac21_up_dwn'].shift(-21)

In [7]:
a.iloc[-1]

o              9.182000e+01
h              9.297000e+01
l              9.050000e+01
c              9.175000e+01
ac             9.175000e+01
v              3.870670e+07
dow            4.000000e+00
vpd_hlc        1.567072e+07
vpd_opac       5.529553e+08
ac1            9.209000e+01
ac3            9.403000e+01
ac5            9.247000e+01
ac8            9.656000e+01
ac13           9.661000e+01
ac21           8.968000e+01
mu_rol3        9.205667e+01
mu_rol5        9.313600e+01
mu_rol8        9.332250e+01
mu_rol13       9.459077e+01
mu_rol21       9.568000e+01
std_rol3       2.914339e-01
std_rol5       1.577842e+00
std_rol8       1.600455e+00
std_rol13      2.225513e+00
std_rol21      2.371683e+00
hl_pct         2.690047e-02
oc_pct        -7.623578e-04
ac1_pct        9.109000e+01
ac3_pct        9.303000e+01
ac5_pct        9.147000e+01
ac8_pct        9.556000e+01
ac13_pct       9.561000e+01
ac21_pct       8.868000e+01
ac1_up_dwn     0.000000e+00
ac3_up_dwn     1.000000e+00
ac5_up_dwn     1.000

In [8]:
a.columns

Index(['o', 'h', 'l', 'c', 'ac', 'v', 'dow', 'vpd_hlc', 'vpd_opac', 'ac1',
       'ac3', 'ac5', 'ac8', 'ac13', 'ac21', 'mu_rol3', 'mu_rol5', 'mu_rol8',
       'mu_rol13', 'mu_rol21', 'std_rol3', 'std_rol5', 'std_rol8', 'std_rol13',
       'std_rol21', 'hl_pct', 'oc_pct', 'ac1_pct', 'ac3_pct', 'ac5_pct',
       'ac8_pct', 'ac13_pct', 'ac21_pct', 'ac1_up_dwn', 'ac3_up_dwn',
       'ac5_up_dwn', 'ac8_up_dwn', 'ac13_up_dwn', 'ac21_up_dwn', 'ac1_pred',
       'ac3_pred', 'ac5_pred', 'ac8_pred', 'ac13_pred', 'ac21_pred'],
      dtype='object')

In [9]:
a1_col_list = ['o'
               # , 'h'
               # , 'l'
               , 'c'
               # ,
               , 'ac'
               # , 'v'
               , 'vpd_hlc'
               , 'vpd_opac'
               # , 'ac1'
               # , 'ac3'
               # , 'ac5'
               # , 'ac8'
               # , 'ac13'
               # , 'ac21'
               , 'mu_rol3'
               # , 'mu_rol5'
               , 'mu_rol8'
               # , 'mu_rol13'
               , 'mu_rol21'
               , 'std_rol3'
               # , 'std_rol5'
               , 'std_rol8'
               # , 'std_rol13'
               , 'std_rol21'
               , 'hl_pct'
               , 'oc_pct'
               , 'ac1_pct'
               # , 'ac3_pct'
               # , 'ac5_pct'
               , 'ac8_pct'
               # , 'ac13_pct'
               , 'ac21_pct'
               # , 'ac1_up_dwn'
               # , 'ac3_up_dwn'
               # , 'ac5_up_dwn'
               # , 'ac8_up_dwn'
               # , 'ac13_up_dwn'
               # , 'ac21_up_dwn'
               , 'ac1_pred'
              ]

a1_master = a[a1_col_list].copy()

a1_X_pred = a1_master[a1_col_list[:-1]].iloc[-1].copy()

a1_master0 = a1_master.iloc[:-1].sample(frac=1, random_state=21)
a1_master0 = a1_master0.sample(frac=1, random_state=42)

idx_split = int(a.shape[0] * 0.9)

a1_X_train = a1_master0.iloc[:idx_split].drop('ac1_pred', axis=1)
a1_y_train = a1_master0['ac1_pred'].iloc[:idx_split]
a1_X_test = a1_master0.iloc[idx_split:].drop('ac1_pred', axis=1)
a1_y_test = a1_master0['ac1_pred'].iloc[idx_split:]

In [10]:
a1_X_pred

o            9.182000e+01
c            9.175000e+01
ac           9.175000e+01
vpd_hlc      1.567072e+07
vpd_opac     5.529553e+08
mu_rol3      9.205667e+01
mu_rol8      9.332250e+01
mu_rol21     9.568000e+01
std_rol3     2.914339e-01
std_rol8     1.600455e+00
std_rol21    2.371683e+00
hl_pct       2.690047e-02
oc_pct      -7.623578e-04
ac1_pct      9.109000e+01
ac8_pct      9.556000e+01
ac21_pct     8.868000e+01
Name: 2023-04-14 00:00:00, dtype: float64

In [11]:
a.iloc[-1]

o              9.182000e+01
h              9.297000e+01
l              9.050000e+01
c              9.175000e+01
ac             9.175000e+01
v              3.870670e+07
dow            4.000000e+00
vpd_hlc        1.567072e+07
vpd_opac       5.529553e+08
ac1            9.209000e+01
ac3            9.403000e+01
ac5            9.247000e+01
ac8            9.656000e+01
ac13           9.661000e+01
ac21           8.968000e+01
mu_rol3        9.205667e+01
mu_rol5        9.313600e+01
mu_rol8        9.332250e+01
mu_rol13       9.459077e+01
mu_rol21       9.568000e+01
std_rol3       2.914339e-01
std_rol5       1.577842e+00
std_rol8       1.600455e+00
std_rol13      2.225513e+00
std_rol21      2.371683e+00
hl_pct         2.690047e-02
oc_pct        -7.623578e-04
ac1_pct        9.109000e+01
ac3_pct        9.303000e+01
ac5_pct        9.147000e+01
ac8_pct        9.556000e+01
ac13_pct       9.561000e+01
ac21_pct       8.868000e+01
ac1_up_dwn     0.000000e+00
ac3_up_dwn     1.000000e+00
ac5_up_dwn     1.000

In [12]:
print(a.shape)
print(idx_split)
print(a1_X_train.shape)
print(a1_y_train.shape)
print(a1_X_test.shape)
print(a1_y_test.shape)

(10841, 45)
9756
(9756, 16)
(9756,)
(1084, 16)
(1084,)


In [13]:
9583+1064

10647

In [14]:
a.shape

(10841, 45)

In [15]:
a1_X_train.iloc[0]

o            9.500000e+00
c            9.843750e+00
ac           9.843750e+00
vpd_hlc      8.863147e+06
vpd_opac     1.208611e+07
mu_rol3      9.593750e+00
mu_rol8      9.402344e+00
mu_rol21     9.111607e+00
std_rol3     2.187500e-01
std_rol8     2.535669e-01
std_rol21    5.084800e-01
hl_pct       4.934211e-02
oc_pct       3.618421e-02
ac1_pct      8.437500e+00
ac8_pct      8.312500e+00
ac21_pct     7.750000e+00
Name: 1999-08-27 00:00:00, dtype: float64

In [16]:
a1_y_train[0]

2.0

In [17]:
a1_X_test.iloc[0]

o            7.010000e+00
c            7.100000e+00
ac           7.100000e+00
vpd_hlc      5.249555e+07
vpd_opac     1.166571e+08
mu_rol3      6.996667e+00
mu_rol8      6.702500e+00
mu_rol21     6.641429e+00
std_rol3     2.514623e-01
std_rol8     2.994160e-01
std_rol21    2.309607e-01
hl_pct       2.853064e-02
oc_pct       1.283876e-02
ac1_pct      6.180000e+00
ac8_pct      5.350000e+00
ac21_pct     5.930000e+00
Name: 2003-07-08 00:00:00, dtype: float64

In [18]:
a1_y_test[0]

0.0

In [19]:
a1_master['ac1_pred'].value_counts()

1.0    4376
2.0    4322
0.0    2142
Name: ac1_pred, dtype: int64

In [20]:
a1_X_train.isin([np.inf, -np.inf]).sum()

o            0
c            0
ac           0
vpd_hlc      0
vpd_opac     0
mu_rol3      0
mu_rol8      0
mu_rol21     0
std_rol3     0
std_rol8     0
std_rol21    0
hl_pct       0
oc_pct       0
ac1_pct      0
ac8_pct      0
ac21_pct     0
dtype: int64

In [31]:
# modeling

xgb_model = xgb.XGBClassifier(objective='binary:logistic'
                              # , num_class=2
                              , random_state=42
                              # , max_depth=7
                              # , n_estimators=89
                              # , learning_rate=0.05
                              # , min_child_weight=4
                             )

xgb_model.fit(a1_X_train, a1_y_train)

y_pred = xgb_model.predict(a1_X_test)

y_pred_prob = xgb_model.predict_proba(a1_X_test)[:, 1]

acc = accuracy_score(a1_y_test, y_pred)

print(f'Accuracy: {acc}')

Accuracy: 0.4077490774907749


In [32]:
xgb_model.feature_importances_

array([0.05487933, 0.06463844, 0.        , 0.06028321, 0.06216793,
       0.0771683 , 0.07325336, 0.06783781, 0.05855045, 0.06655625,
       0.06788228, 0.06882254, 0.06117975, 0.08007969, 0.06914838,
       0.06755223], dtype=float32)

In [33]:
a1_X_train.columns

Index(['o', 'c', 'ac', 'vpd_hlc', 'vpd_opac', 'mu_rol3', 'mu_rol8', 'mu_rol21',
       'std_rol3', 'std_rol8', 'std_rol21', 'hl_pct', 'oc_pct', 'ac1_pct',
       'ac8_pct', 'ac21_pct'],
      dtype='object')

In [34]:
xgb_model.predict(a1_X_pred.to_numpy().reshape(1,-1))[0]

1

In [35]:
xgb_model.predict_proba(a1_X_pred.to_numpy().reshape(1,-1))

array([[0.15556449, 0.6014525 , 0.24298297]], dtype=float32)

In [36]:
type(a1_X_pred)

pandas.core.series.Series

In [37]:
a1_X_pred.shape

(16,)

In [38]:
# add predictions to X_train

a1_master1 = a[a1_col_list].copy()
a1_master1['pred0'] = xgb_model.predict(a1_master1[a1_X_train.columns])

In [39]:
a1_master1.tail(10)

,o,c,ac,vpd_hlc,vpd_opac,mu_rol3,mu_rol8,mu_rol21,std_rol3,std_rol8,std_rol21,hl_pct,oc_pct,ac1_pct,ac8_pct,ac21_pct,ac1_pred,pred0
Date,,,,,,,,,,,,,,,,,,
2023-03-31,96.339996,98.010002,98.010002,1.806716e+07,3.342958e+07,97.326665,97.369999,91.530475,1.072958,1.674618,7.018199,0.032074,0.017335,96.879997,94.930000,79.440002,1.0,1
2023-04-03,96.699997,96.559998,96.559998,2.558344e+07,3.618251e+08,97.483332,97.242498,92.246666,0.802269,1.695050,6.706032,0.020476,-0.001448,97.010002,96.580002,80.519997,1.0,1
2023-04-04,97.040001,95.870003,95.870003,2.096891e+07,3.691963e+07,96.813334,96.691249,92.947142,1.092261,1.215301,6.242306,0.021228,-0.012057,95.559998,99.279999,80.160004,1.0,1
2023-04-05,94.349998,92.559998,92.559998,1.687239e+07,2.959736e+07,94.996666,96.017499,93.444761,2.138232,1.780448,5.730767,0.033280,-0.018972,94.870003,96.949997,81.110001,0.0,0
2023-04-06,91.470001,92.470001,92.470001,2.086392e+07,4.777840e+07,93.633334,95.499999,93.782856,1.937535,2.147464,5.432227,0.025036,0.010933,91.559998,95.610001,84.370003,2.0,2
2023-04-10,91.320000,95.480003,95.480003,1.099084e+07,1.262891e+07,93.503334,95.615000,94.328094,1.712437,2.114312,4.958331,0.052343,0.045554,91.470001,93.559998,83.029999,1.0,1
2023-04-11,96.059998,94.029999,94.029999,1.536563e+07,2.558415e+07,93.993334,95.357500,94.869047,1.505336,2.172831,4.181706,0.035186,-0.021133,94.480003,95.089996,81.669998,1.0,1
2023-04-12,94.970001,92.330002,92.330002,1.637517e+07,2.022088e+07,93.946668,94.663751,95.360475,1.576653,2.138117,3.047569,0.034327,-0.027798,93.029999,96.879997,81.010002,0.0,0
2023-04-13,92.790001,92.089996,92.089996,3.050560e+07,5.796034e+07,92.816666,93.923750,95.581428,1.057607,1.814497,2.577292,0.014333,-0.007544,91.330002,97.010002,86.449997,0.0,0


In [45]:
a1_master1.shape

(10841, 18)

In [40]:
len(a1_X_train.columns)

16

In [46]:
a1_X_columns = list(a1_X_train.columns)
a1_X_columns.append('pred0')
a1_X_columns

['o',
 'c',
 'ac',
 'vpd_hlc',
 'vpd_opac',
 'mu_rol3',
 'mu_rol8',
 'mu_rol21',
 'std_rol3',
 'std_rol8',
 'std_rol21',
 'hl_pct',
 'oc_pct',
 'ac1_pct',
 'ac8_pct',
 'ac21_pct',
 'pred0']

In [48]:
a1_X_pred1 = a1_master1[a1_X_columns].iloc[-1].copy()
a1_X_pred1

o            9.182000e+01
c            9.175000e+01
ac           9.175000e+01
vpd_hlc      1.567072e+07
vpd_opac     5.529553e+08
mu_rol3      9.205667e+01
mu_rol8      9.332250e+01
mu_rol21     9.568000e+01
std_rol3     2.914339e-01
std_rol8     1.600455e+00
std_rol21    2.371683e+00
hl_pct       2.690047e-02
oc_pct      -7.623578e-04
ac1_pct      9.109000e+01
ac8_pct      9.556000e+01
ac21_pct     8.868000e+01
pred0        1.000000e+00
Name: 2023-04-14 00:00:00, dtype: float64

In [49]:
a1_master1_scrambled = a1_master1.iloc[:-1].copy()
a1_master1_scrambled.tail(10)

,o,c,ac,vpd_hlc,vpd_opac,mu_rol3,mu_rol8,mu_rol21,std_rol3,std_rol8,std_rol21,hl_pct,oc_pct,ac1_pct,ac8_pct,ac21_pct,ac1_pred,pred0
Date,,,,,,,,,,,,,,,,,,
2023-03-30,98.000000,97.879997,97.879997,2.617479e+07,4.973095e+08,96.176664,97.109999,90.693809,1.661696,1.721860,7.250574,0.023265,-0.001225,95.089996,95.809998,77.290001,0.0,0
2023-03-31,96.339996,98.010002,98.010002,1.806716e+07,3.342958e+07,97.326665,97.369999,91.530475,1.072958,1.674618,7.018199,0.032074,0.017335,96.879997,94.930000,79.440002,1.0,1
2023-04-03,96.699997,96.559998,96.559998,2.558344e+07,3.618251e+08,97.483332,97.242498,92.246666,0.802269,1.695050,6.706032,0.020476,-0.001448,97.010002,96.580002,80.519997,1.0,1
2023-04-04,97.040001,95.870003,95.870003,2.096891e+07,3.691963e+07,96.813334,96.691249,92.947142,1.092261,1.215301,6.242306,0.021228,-0.012057,95.559998,99.279999,80.160004,1.0,1
2023-04-05,94.349998,92.559998,92.559998,1.687239e+07,2.959736e+07,94.996666,96.017499,93.444761,2.138232,1.780448,5.730767,0.033280,-0.018972,94.870003,96.949997,81.110001,0.0,0
2023-04-06,91.470001,92.470001,92.470001,2.086392e+07,4.777840e+07,93.633334,95.499999,93.782856,1.937535,2.147464,5.432227,0.025036,0.010933,91.559998,95.610001,84.370003,2.0,2
2023-04-10,91.320000,95.480003,95.480003,1.099084e+07,1.262891e+07,93.503334,95.615000,94.328094,1.712437,2.114312,4.958331,0.052343,0.045554,91.470001,93.559998,83.029999,1.0,1
2023-04-11,96.059998,94.029999,94.029999,1.536563e+07,2.558415e+07,93.993334,95.357500,94.869047,1.505336,2.172831,4.181706,0.035186,-0.021133,94.480003,95.089996,81.669998,1.0,1
2023-04-12,94.970001,92.330002,92.330002,1.637517e+07,2.022088e+07,93.946668,94.663751,95.360475,1.576653,2.138117,3.047569,0.034327,-0.027798,93.029999,96.879997,81.010002,0.0,0


In [51]:
a1_master1_scrambled = a1_master1_scrambled.sample(frac=1)
a1_master1_scrambled = a1_master1_scrambled.sample(frac=1)

a1_master1_scrambled.tail(10)

,o,c,ac,vpd_hlc,vpd_opac,mu_rol3,mu_rol8,mu_rol21,std_rol3,std_rol8,std_rol21,hl_pct,oc_pct,ac1_pct,ac8_pct,ac21_pct,ac1_pred,pred0
Date,,,,,,,,,,,,,,,,,,
2000-07-05,37.50000,37.62500,37.62500,6.879306e+06,1.427456e+08,39.458333,40.808594,42.809524,2.362908,2.481991,2.627360,0.069167,0.003333,41.12500,44.62500,44.56250,2.0,2
2014-11-21,2.72000,2.77000,2.77000,1.885268e+08,2.262322e+08,2.696667,2.673750,2.728571,0.070238,0.050125,0.069303,0.022059,0.018382,1.69000,1.73000,1.69000,2.0,2
1988-12-05,4.00000,3.93750,3.93750,3.492000e+06,1.396800e+07,3.958333,3.812500,3.931548,0.036084,0.133631,0.226992,0.062500,-0.015625,2.93750,2.81250,3.37500,1.0,1
1995-02-03,15.75000,15.68750,15.68750,1.159307e+07,6.955840e+07,15.416667,15.156250,15.264881,0.281828,0.311606,0.608823,0.023810,-0.003968,14.43750,14.62500,11.75000,2.0,2
2007-07-25,15.38000,15.58000,15.58000,9.170665e+07,1.971692e+08,15.383333,15.556250,14.940000,0.186100,0.216725,0.596724,0.027958,0.013004,14.21000,14.43000,13.28000,1.0,2
1991-06-13,6.31250,6.37500,6.37500,5.661440e+06,2.830720e+07,6.500000,6.640625,6.389881,0.216506,0.215861,0.460089,0.049505,0.009901,5.37500,6.00000,4.93750,2.0,2
1983-03-17,0.00000,8.78125,8.78125,1.569600e+06,4.468612e+04,8.854167,8.527344,8.821429,0.100455,0.289047,0.383897,0.000000,0.000000,7.96875,7.71875,8.34375,2.0,1
1993-10-27,9.75000,9.93750,9.93750,6.972800e+06,1.394560e+07,9.833333,9.835938,10.702381,0.130104,0.203039,1.384612,0.038462,0.019231,8.68750,9.12500,14.00000,0.0,0
1983-08-04,14.56250,14.18750,14.18750,5.318400e+05,8.864000e+05,14.447917,15.074219,15.404762,0.238676,0.793201,0.867010,0.042918,-0.025751,13.50000,15.46875,14.90625,0.0,0


In [54]:
a1_idx_tt_split = int(a1_master1_scrambled.shape[0] * 0.9)
a1_idx_tt_split

9756

In [57]:
a1_X_train = a1_master1_scrambled[a1_X_columns].iloc[:a1_idx_tt_split]
a1_y_train = a1_master1_scrambled['ac1_pred'].iloc[:a1_idx_tt_split]

a1_X_test = a1_master1_scrambled[a1_X_columns].iloc[a1_idx_tt_split:]
a1_y_test = a1_master1_scrambled['ac1_pred'].iloc[a1_idx_tt_split:]

print(f'a1_master1 shape: {a1_master1.shape}')
print(f'X_train shape: {a1_X_train.shape}')
print(f'y_train shape: {a1_y_train.shape}')
print(f'X_test shape: {a1_X_test.shape}')
print(f'y_test shape: {a1_y_test.shape}')
print(f'a1_X_pred1 shape: {a1_X_pred1.shape}')

a1_master1 shape: (10841, 18)
X_train shape: (9756, 17)
y_train shape: (9756,)
X_test shape: (1084, 17)
y_test shape: (1084,)
a1_X_pred1 shape: (17,)


In [59]:
# 2nd modeling

xgb_model1 = xgb.XGBClassifier(objective='binary:logistic'
                              # , num_class=2
                              , random_state=42
                              # , max_depth=7
                              # , n_estimators=89
                              # , learning_rate=0.05
                              # , min_child_weight=4
                             )

xgb_model1.fit(a1_X_train, a1_y_train)

y_pred = xgb_model1.predict(a1_X_test)

y_pred_prob = xgb_model1.predict_proba(a1_X_test)[:, 1]

acc = accuracy_score(a1_y_test, y_pred)

print(f'Accuracy: {acc}')

Accuracy: 0.827490774907749


In [60]:
xgb_model1.feature_importances_

array([0.01964513, 0.02157363, 0.        , 0.02048212, 0.01939854,
       0.02268316, 0.0237458 , 0.02281511, 0.0195689 , 0.0189837 ,
       0.01952966, 0.01982659, 0.01951261, 0.02462604, 0.0212805 ,
       0.0203747 , 0.6859538 ], dtype=float32)

In [61]:
a1_X_columns

['o',
 'c',
 'ac',
 'vpd_hlc',
 'vpd_opac',
 'mu_rol3',
 'mu_rol8',
 'mu_rol21',
 'std_rol3',
 'std_rol8',
 'std_rol21',
 'hl_pct',
 'oc_pct',
 'ac1_pct',
 'ac8_pct',
 'ac21_pct',
 'pred0']